# Amazon Copurchased

This is a Python notebook created using "jupyter".

Author: Rafael J. P. dos Santos

## Parameters

We use the parameter below to set the maximum number of edges to be read from the CSV containing edges (links).

In [1]:
max_edges = 1000 # Set to 0 to read all nodes from file
edges_csv_file = "data/20180810_links"
nodes_csv_file = "data/20180810_nodes"

## Load the libraries

Let's load the Python libraries that we will need throughout the script

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import export_graphviz
import pydot
from sklearn.ensemble import RandomForestRegressor

## Read graph

### Read only first lines of datafile

Due to slowness in calculating centrality measures, we use the parameter provided in the beggining of the script to limit the number of edges we will read.

In [3]:
lines = []
with open(edges_csv_file, 'rb') as f:
    f.readline()   # skip first line / header
    while True:
        line = f.readline()
        if not line or (max_edges > 0 and len(lines) >= max_edges):
            break
        lines.append(line)
G = nx.parse_edgelist(lines, delimiter=',', nodetype=int)

### Calculate nodes centrality measures

Now that we have our NetworkX graph, let's calculate some centrality measures for every node.

#### Degree

In [4]:
degrees = nx.degree(G)

#### Eigenvector centrality

In [5]:
ec = nx.eigenvector_centrality(G)

#### Closeness centrality

In [6]:
# Very slow!
cc = nx.closeness_centrality(G)

#### Betweenness centrality

In [7]:
# Very slow!
bc = nx.betweenness_centrality(G)

### Load node properties

Let's load the CSV containing the nodes data (title, price) into a Pandas dataframe, and append the centrality measures calculated above.

In [8]:
df = pd.read_csv(nodes_csv_file)
df['degree'] = None
df['eigenvector_centrality'] = None
df['closeness_centrality'] = None
df['betweenness_centrality'] = None
for index, row in df.iterrows():
    try:
        df.loc[index, 'degree'] = degrees[row['id']]
        df.loc[index, 'eigenvector_centrality'] = ec[row['id']]
        df.loc[index, 'closeness_centrality'] = cc[row['id']]
        df.loc[index, 'betweenness_centrality'] = bc[row['id']]
    except:
        df.drop([index], inplace=True)

### Remove nodes without price and outliers

In [76]:
df = df.drop(df[df["price"].isnull()].index)
df = df.drop(df[df["price"] > 300].index)

### Let's convert some fields to numeric

In [77]:
categorical_features = [
    'category1',
    'category2',
    'category3',
    'language',
    'coverType',
    'publisher',
    'rankingCategory'
]

numeric_features = [
    'degree',
    'eigenvector_centrality',
    'closeness_centrality',
    'betweenness_centrality',
    'ranking',
    'reviewCount',
    'pages',
    'weight',
    'height',
    'width',
    'depth',
    'rating'
]

df = df.replace("<<MISSING_DATA>>", np.NaN)
df[numeric_features] = df[numeric_features].apply(pd.to_numeric)
df[['price']] = df[['price']].apply(pd.to_numeric)

for feature in numeric_features:
    df[feature].fillna(df[feature].mean(), inplace = True)

### Inspect columns

In [78]:
df.columns

Index([u'id', u'title', u'url', u'authors', u'coverType', u'publisher',
       u'edition', u'publicationDate', u'rankingCategory', u'category1',
       u'category2', u'category3', u'category4', u'category5', u'category6',
       u'category7', u'category8', u'category9', u'category10', u'isbn10',
       u'isbn13', u'language', u'postProcessed', u'price', u'ranking',
       u'pages', u'reviewCount', u'rating', u'width', u'height', u'depth',
       u'weight', u'degree', u'eigenvector_centrality',
       u'closeness_centrality', u'betweenness_centrality'],
      dtype='object')

### Features summary

Below we have a summary of the Pandas dataframe. We can see the number of nodes that we are actually analyzing, which depends on the max_edges parameter defined before.

In [79]:
df.describe(include='all')

,id,title,url,authors,coverType,publisher,edition,publicationDate,rankingCategory,category1,...,reviewCount,rating,width,height,depth,weight,degree,eigenvector_centrality,closeness_centrality,betweenness_centrality
count,375.000000,375,375,375,373,373,0.0,57,366,366,...,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000
unique,NaN,363,375,299,2,97,NaN,56,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Cálculo - Volume 2,https://www.amazon.com.br/dp/8577804828/,"Richard P. Feynman (Autor),",Capa comum,Dover Publications,NaN,27 de agosto de 2018,Livros,Livros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,1,6,345,56,NaN,2,366,366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,775.485333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.309290,4.582593,16.332356,23.407024,2.174415,461.237480,4.320000,0.023805,0.270281,0.006517
std,1252.259976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.445275,0.444681,2.718732,2.592803,1.033565,208.744110,7.745898,0.040681,0.033856,0.023048
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.900000,10.200000,14.800000,0.400000,59.000000,1.000000,0.000335,0.201346,0.000000
25%,108.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,4.592276,14.000000,21.400000,1.400000,322.000000,1.000000,0.003075,0.251199,0.000000
50%,267.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.260163,4.592276,15.800000,22.900000,2.000000,464.027987,1.000000,0.007483,0.269627,0.000000
75%,457.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.260163,5.000000,17.800000,24.650000,2.800000,574.000000,3.000000,0.021563,0.288369,0.000645


Below we can inspect the first rows of data, containing title, price, degree and other centrality measures.

In [80]:
df.head(10)

,id,title,url,authors,coverType,publisher,edition,publicationDate,rankingCategory,category1,...,reviewCount,rating,width,height,depth,weight,degree,eigenvector_centrality,closeness_centrality,betweenness_centrality
0,1,The Stanford Mathematics Problem Book: With Hi...,https://www.amazon.com.br/dp/0486469247/,"George Polya (Autor),",Capa comum,Dover Publications,NaN,19 de fevereiro de 2009,Livros,Livros,...,1.000000,4.000000,14.0,21.0,0.6,181.0,15,0.092519,0.343443,0.073199
1,2,Fourier Series,https://www.amazon.com.br/dp/0486633179/,"Georgi P. Tolstov (Autor),",Capa comum,Dover Publications,NaN,1 de junho de 1976,Livros,Livros,...,3.000000,4.600000,14.6,21.0,1.9,363.0,21,0.173882,0.344007,0.029658
2,3,Probability Theory: A Concise Course,https://www.amazon.com.br/dp/0486635449/,"Y. A. Rozanov (Autor),",Capa comum,Dover Publications,NaN,NaN,Livros,Livros,...,6.260163,4.592276,14.4,20.8,0.8,200.0,39,0.243070,0.335200,0.089836
3,4,"Vectors, Tensors and the Basic Equations of Fl...",https://www.amazon.com.br/dp/0486661105/,"Rutherford Aris (Autor),",Capa comum,Dover Publications,NaN,NaN,Livros,Livros,...,2.000000,4.500000,13.7,21.5,1.6,381.0,10,0.074681,0.312221,0.005997
4,5,Números. Uma Introdução à Matemática,https://www.amazon.com.br/dp/8531404584/,"César Polcino Milies (Autor),",Capa comum,EDUSP,NaN,NaN,Livros,Livros,...,6.260163,4.592276,15.8,22.8,1.6,440.0,13,0.023458,0.346854,0.124833
5,6,Ordinary Differential Equations,https://www.amazon.com.br/dp/0486649407/,"Morris Tenenbaum (Autor),",Capa comum,Dover Publications,NaN,NaN,Livros,Livros,...,4.000000,4.500000,13.8,21.8,4.0,939.0,34,0.244177,0.334932,0.057468
6,7,The Variational Principles of Mechanics,https://www.amazon.com.br/dp/0486650677/,"Cornelius Lanczos (Autor),",Capa comum,Dover Publications,NaN,NaN,Livros,Livros,...,1.000000,5.000000,13.8,21.5,2.2,581.0,26,0.216816,0.329662,0.026434
7,8,A First Look at Perturbation Theory,https://www.amazon.com.br/dp/0486675513/,James G. Simmonds (Autor),Capa comum,Dover Publications Inc.,NaN,NaN,Livros,Livros,...,6.260163,4.592276,13.7,21.5,0.8,159.0,3,0.023477,0.264187,0.000023
9,10,Geometria Analítica,https://www.amazon.com.br/dp/0074504096/,"Alfredo Steimbruch (Autor),",Capa comum,Pearson,NaN,NaN,Livros,Livros,...,1.000000,5.000000,15.6,23.0,1.4,240.0,24,0.031744,0.344856,0.048665
10,11,Curso de Física Básica: Eletromagnetismo (Volu...,https://www.amazon.com.br/dp/8521208014/,Herch Moysés Nussenzveig (Autor),Capa comum,Blucher,NaN,NaN,Livros,Livros,...,4.000000,4.100000,16.8,23.9,1.5,522.0,26,0.041158,0.337088,0.038658


## Random forest using degree as feature, price as target

### Preparing data

In [81]:
df_with_dummies = pd.get_dummies(df[numeric_features + categorical_features + ['price']],columns=categorical_features,drop_first=True)

In [82]:
feature_list = list(df_with_dummies.drop(columns = ['price']))
features = np.array(df_with_dummies.drop(columns = ['price']))
target = np.array(df_with_dummies['price'])

### Average price as baseline

It's important to have a baseline, so we can validate our predictions after running our model. One easy choice for baseline is the average price of a book.

We have an average price around R\$43, so this means that a very easy prediction would be to always guess R$43 for the price of any book.

In [83]:
average_target = np.average(target)
print "Average price: R$", average_target

Average price: R$ 79.3456533333


### Training data split

Let's split our dataset into two sets: train and test. We use the first to train or model, and we use the second to test the precision of our model.

In [84]:
# Split the data into training and testing sets
train_features, test_features, train_target, test_target \
    = train_test_split(features, target, test_size = 0.25)

# Summary
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_target.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_target.shape)

('Training Features Shape:', (281, 160))
('Training Labels Shape:', (281,))
('Testing Features Shape:', (94, 160))
('Testing Labels Shape:', (94,))


### Train data

We train a random forest model with 500 estimators.

In [85]:
# Load model
rf = RandomForestRegressor(n_estimators = 500)
# Train
rf.fit(train_features, train_target);

### Prediction

In [86]:
predictions = rf.predict(test_features)

### Mean absolute error

Now we can compare the errors obtained by our predictions against the errors provided by the baseline (average price). Our prediction errors should be less than the baseline errors to consider the model successful.

In [87]:
# Calculate the absolute errors
errors = abs(predictions - test_target)
errors_baseline = abs(average_target - test_target)
# Print out the mean absolute error (mae)
print('Mean absolute prediction error: R$', round(np.mean(errors), 2))
print('Mean absolute error using average: R$',
      round(np.mean(errors_baseline), 2))

('Mean absolute prediction error: R$', 23.34)
('Mean absolute error using average: R$', 43.02)


### Worst predictions

Below we can inspect the rows with the biggest prediction error.

In [88]:
data = {
    "degree": test_features.tolist(),
    "target": test_target,
    "prediction": predictions,
    "error": errors,
    "errors_baseline": errors_baseline
}
predicted_df = pd.DataFrame(data = data)
predicted_df.sort_values('error', ascending = False).head(10)

,degree,error,errors_baseline,prediction,target
35,"[3.0, 0.0491248329086, 0.288965517241, 7.18038...",158.17190,195.314347,116.48810,274.66
89,"[3.0, 0.0334339887261, 0.294448348559, 0.00148...",139.69454,203.534347,143.18546,282.88
62,"[4.0, 0.0656190007526, 0.309453471196, 0.00106...",83.65130,160.444347,156.13870,239.79
52,"[2.0, 0.0240122143008, 0.276021080369, 0.00035...",82.27428,5.495653,156.12428,73.85
13,"[2.0, 0.0174887696654, 0.290972222222, 0.0, 57...",78.09924,31.445653,125.99924,47.90
16,"[1.0, 0.00339131956247, 0.270322580645, 0.0, 4...",76.00166,57.574347,60.91834,136.92
21,"[1.0, 0.00355543342375, 0.253478523896, 0.0, 1...",64.73756,86.204347,100.81244,165.55
55,"[1.0, 0.00307532148158, 0.269626769627, 0.0, 6...",60.60130,114.554347,133.29870,193.90
81,"[1.0, 0.00294479344659, 0.248517200474, 0.0, 5...",59.43118,108.884347,128.79882,188.23
7,"[8.0, 0.0270820321774, 0.327599687256, 0.00866...",55.68664,49.354347,73.01336,128.70


### Best predictions

In [92]:
predicted_df.sort_values('error', ascending = True).head(10)

,degree,error,errors_baseline,prediction,target
41,"[1.0, 0.00283150298944, 0.252409638554, 0.0, 4...",0.06044,36.445653,42.83956,42.90
75,"[1.0, 0.000537978146318, 0.227346717309, 0.0, ...",0.79838,24.445653,55.69838,54.90
36,"[1.0, 0.000335251387034, 0.222872340426, 0.0, ...",0.86614,25.464347,103.94386,104.81
78,"[1.0, 0.00283150298944, 0.252409638554, 0.0, 1...",0.98046,23.594347,103.92046,102.94
19,"[1.0, 0.000537978146318, 0.227346717309, 0.0, ...",2.20536,28.445653,53.10536,50.90
14,"[2.0, 0.0113678478101, 0.286006825939, 0.0, 89...",2.28346,39.385653,37.67654,39.96
32,"[2.0, 0.00606049829833, 0.28659370725, 0.00037...",2.30276,40.644347,117.68724,119.99
67,"[1.0, 0.0123372841329, 0.270671834625, 0.0, 26...",2.33198,10.835653,70.84198,68.51
33,"[1.0, 0.000335251387034, 0.222872340426, 0.0, ...",2.61076,0.345653,76.38924,79.00
20,"[14.0, 0.132975907683, 0.308769344141, 0.01138...",2.72556,49.295653,32.77556,30.05


In [93]:
predicted_df.describe()

,error,errors_baseline,prediction,target
count,94.000000,94.000000,94.000000,94.000000
mean,23.338825,43.024316,80.405272,86.204362
std,27.216440,38.985623,38.006300,57.821511
min,0.060440,0.345653,31.553200,14.900000
25%,7.470910,20.943153,51.707330,48.167500
50%,14.352060,32.970653,72.773310,65.970000
75%,30.583430,49.595327,104.769430,118.957500
max,158.171900,203.534347,189.418500,282.880000


### Visualize decision tree

In [94]:
# Pull out one tree from the forest
tree = rf.estimators_[0]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot',
                feature_names = feature_list, rounded = True)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

<img src="files/image.png">